## Test spark

In [1]:
import os
import inspect
# make sure pyspark tells workers to use python3 not 2 if both are installed
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'


import pyspark
conf = pyspark.SparkConf()
print ("t1")

print (1)
# create the context
sc = pyspark.SparkContext(conf=conf)
print("after sc")
# do something to prove it works
rdd = sc.parallelize([1,2,3,4])
print("after rdd")

# collect the RDD to a list
list_elements = rdd.collect()
print("after collect")

# print the list
for element in list_elements:
    print(element)
count = 0

t1
1
after sc
after rdd
after collect
1
2
3
4


In [2]:
%%sql
%defaultDatasource jdbc:splice://agent-000001:1527/splicedb;user=splice;password=kolj$6drornUrk

Sql started successfully



## Test PySplice creation

In [3]:
import pyspark
from splicemachine.spark.context import PySpliceContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import os
os.environ['JDBC_HOST'] = 'agent-000001'
jdbc_host = os.environ['JDBC_HOST']

conf = pyspark.SparkConf()

#spark=SparkSession.builder.config(conf=SparkConf())
spark = SparkSession.builder.config(conf=conf).getOrCreate()

splicejdbc=f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk'

splice = PySpliceContext(spark, splicejdbc)

# Query Features
query_results = splice.df("select * from sys.systables")
query_results.show(10)

+--------------------+--------------------+---------+--------------------+---------------+-------+-----------+---------+-------+-----+------+--------+-----------+---------+------------------+
|             TABLEID|           TABLENAME|TABLETYPE|            SCHEMAID|LOCKGRANULARITY|VERSION|COLSEQUENCE|DELIMITED|ESCAPED|LINES|STORED|LOCATION|COMPRESSION|IS_PINNED|PURGE_DELETED_ROWS|
+--------------------+--------------------+---------+--------------------+---------------+-------+-----------+---------+-------+-----+------+--------+-----------+---------+------------------+
|00cd0029-016c-625...|GLOBAL_LOCATION_I...|        T|08264012-016c-625...|              R|    4.0|          2|     null|   null| null|  null|    null|       none|    false|             false|
|6da849c2-016d-470...|      TRANSFERORDERS|        T|3f46c48e-016d-470...|              R|    4.0|         19|     null|   null| null|  null|    null|       none|    false|             false|
|38c30028-016d-470...|                  

## All available PySplice functions

In [4]:
# functions = []
# auth_functions = []
for i in dir(splice):
    if i[0] != '_' and i.upper() != i and i not in ['jvm','toUpper', 'replaceDataframeSchema', 'context','jdbcurl']:
        print(i)
#         functions.append(i)
# count = 0                                                    

analyzeSchema
analyzeTable
createTable
delete
df
dropTable
execute
executeUpdate
export
exportBinary
getConnection
getSchema
insert
internalDf
spark_sql_context
tableExists
truncateTable
update
upsert


In [5]:
%%sql
-- drop table if exists foo;
-- create table foo(a int, b varchar(50), c int, primary key(a));
-- insert into foo values(1, 'test', 2);
-- insert into foo values(2, 'foo', 3);
-- insert into foo values(3, 'bar', 12);
-- insert into foo values(4, 'name', 42);
-- insert into foo values(5, 'field', 34);
-- insert into foo values(6, 'nan', 77);

-- select * from splice.foo;

java.sql.SQLSyntaxErrorException:  Syntax error

## Test analyzeSchema

In [6]:
print('Function Definition:')
print(inspect.getsource(splice.analyzeSchema))
print('----------------------------Testing Function---------------------------------\n')
#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    x = splice.analyzeSchema('splice')
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    x = splice.analyzeSchema('splice')
print(type(x))
print(x)
count+=1

Function Definition:
    def analyzeSchema(self, schema_name):
        """
        analyze the schema
        :param schema_name: schema name which stats info will be collected
        :return:
        """
        return self.context.analyzeSchema(schema_name)

----------------------------Testing Function---------------------------------

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reason: userid or password invalid.
<class 'NoneType'>
None


## Test analyzeTable

In [7]:
print('Function Definition:')
print(inspect.getsource(splice.analyzeTable))
print('----------------------------Testing Function---------------------------------\n')

# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo')
splice.execute('create table foo(a int, b varchar(50), c int, primary key(a))')
splice.execute('insert into foo values(1, \'test\', 2)')


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    x = splice.analyzeTable('splice.foo')
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    x = splice.analyzeTable('splice.foo')
print(type(x))
print(x)
count+=1

Function Definition:
    def analyzeTable(self, schema_table_name, estimateStatistics=False, samplePercent=0.10):
        """
        collect stats info on a table
        :param schema_table_name: full table name in the format of "schema.table"
        :param estimateStatistics:will use estimate statistics if True
        :param samplePercent:  the percentage or rows to be sampled.
        :return:
        """
        return self.context.analyzeTable(schema_table_name, estimateStatistics, samplePercent)

----------------------------Testing Function---------------------------------

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reason: userid or password invalid.
<class 'NoneType'>
None


## Test createTable: CAN'T test yet: Bug was found and fixed in DBAAS-2869

In [8]:
# print('Function Definition:')
# print(inspect.getsource(splice.createTable))
# print('----------------------------Testing Function---------------------------------\n')

# if splice: del splice
# try:
#     splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
#     x = splice.df('select * from splice.foo')
#     splice.createTable(x, 'splice.foo1')
# except Exception as e:
#     auth_functions.append(functions[count])
#     print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
#     splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
#     x = splice.df('select * from splice.foo')
#     splice.createTable(x, 'splice.foo1')
# print(type(x))
# print(x)
# count+=1

## Test delete: Note: table must have primary key to call a delete

In [9]:
print('Function Definition:')
print(inspect.getsource(splice.delete))
print('----------------------------Testing Function---------------------------------\n')

# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo')
splice.execute('create table foo(a int, b varchar(50), c int, primary key(a))')
splice.execute('insert into foo values(1, \'test\', 2)')
splice.execute('insert into foo values(2, \'foo\', 3)')
splice.execute('insert into foo values(3, \'bar\', 12)')
splice.execute('insert into foo values(4, \'name\', 42)')
splice.execute('insert into foo values(5, \'field\', 34)')
splice.execute('insert into foo values(6, \'nan\', 77)')
splice.df('select * from foo').show()


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    splice.df('select * from splice.foo').show()
    to_del = splice.df('select * from splice.foo where a=1')
    x = splice.delete(to_del,'splice.foo')
    splice.df('select * from splice.foo').show()
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    to_del = splice.df('select * from splice.foo where a=1')
    x = splice.delete(to_del,'splice.foo')
    splice.df('select * from splice.foo').show()
print(type(x))
print(x)
count+=1

Function Definition:
    def delete(self, dataframe, schema_table_name):
        """
        Delete records in a dataframe based on joining by primary keys from the data frame.
        Be careful with column naming and case sensitivity.

        :param dataframe: (DF) The dataframe you would like to delete
        :param schema_table_name: (string) Splice Machine Table
        """
        return self.context.delete(dataframe._jdf, schema_table_name)

----------------------------Testing Function---------------------------------

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reaso

## Test df

In [10]:
print('Function Definition:')
print(inspect.getsource(splice.df))
print('----------------------------Testing Function---------------------------------\n')


# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo')
splice.execute('create table foo(a int, b varchar(50), c int, primary key(a))')
splice.execute('insert into foo values(1, \'test\', 2)')
splice.execute('insert into foo values(2, \'foo\', 3)')
splice.execute('insert into foo values(3, \'bar\', 12)')
splice.execute('insert into foo values(4, \'name\', 42)')
splice.execute('insert into foo values(5, \'field\', 34)')
splice.execute('insert into foo values(6, \'nan\', 77)')
splice.df('select * from foo').show()


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    x = splice.df('select * from splice.foo where a=3')
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    x = splice.df('select * from splice.foo where a=3')
print(type(x))
print(x)
x.show()
count+=1

Function Definition:
    def df(self, sql):
        """
        Return a Spark Dataframe from the results of a Splice Machine SQL Query

        :param sql: (string) SQL Query (eg. SELECT * FROM table1 WHERE column2 > 3)
        :return: A Spark DataFrame containing the results
        """
        if self._unit_testing:
            return self.context.df(sql)
        return DataFrame(self.context.df(sql), self.spark_sql_context)

----------------------------Testing Function---------------------------------

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

<class 'pyspark.sql.dataframe.DataFrame'>
DataFrame[A: int, B: string, C: int]
+---+---+---+
|  A|  B|  C|
+---+---+---+
|  3|bar| 12|
+---+---+---+



## Test dropTable

In [11]:
print('Function Definition:')
print(inspect.getsource(splice.delete))
print('----------------------------Testing Function---------------------------------\n')



# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo')
splice.execute('create table foo(a int, b varchar(50), c int, primary key(a))')
splice.execute('insert into foo values(1, \'test\', 2)')
splice.execute('insert into foo values(2, \'foo\', 3)')
splice.execute('insert into foo values(3, \'bar\', 12)')
splice.execute('insert into foo values(4, \'name\', 42)')
splice.execute('insert into foo values(5, \'field\', 34)')
splice.execute('insert into foo values(6, \'nan\', 77)')
splice.df('select * from foo').show()


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    splice.df('select * from splice.foo').show()
    splice.dropTable('splice.foo')
    try:
        print('attempting to read post drop')
        splice.df('select * from splice.foo').show()
    except:
        print('drop successful')
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    splice.dropTable('splice.foo')
    try:
        print('attempting to read post drop')
        x = splice.df('select * from splice.foo')
    except:
        print('drop successful')



count+=1

Function Definition:
    def delete(self, dataframe, schema_table_name):
        """
        Delete records in a dataframe based on joining by primary keys from the data frame.
        Be careful with column naming and case sensitivity.

        :param dataframe: (DF) The dataframe you would like to delete
        :param schema_table_name: (string) Splice Machine Table
        """
        return self.context.delete(dataframe._jdf, schema_table_name)

----------------------------Testing Function---------------------------------

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reaso

## Test execute

In [12]:
print('Function Definition:')
print(inspect.getsource(splice.execute))
print('----------------------------Testing Function---------------------------------\n')
#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    splice.execute('drop table if exists splice.foo_two')
    splice.execute('create table splice.foo_two(a int)')
    splice.execute('insert into splice.foo_two values(1)')
    splice.df('select * from splice.foo_two').show()
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    splice.execute('drop table if exists splice.foo_two')
    splice.execute('create table splice.foo_two(a int)')
    splice.execute('insert into splice.foo_two values(1)')
    splice.df('select * from splice.foo_two').show()
    
#     x.show()
# print(type(x))
# print(x)
count+=1

Function Definition:
    def execute(self, query_string):
        '''
        execute a query
        :param query_string: (string) SQL Query (eg. SELECT * FROM table1 WHERE column2 > 3)
        :return:
        '''
        return self.context.execute(query_string)

----------------------------Testing Function---------------------------------

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reason: userid or password invalid.
+---+
|  A|
+---+
|  1|
+---+



## Test executeUpdate

In [13]:
print('Function Definition:')
print(inspect.getsource(splice.executeUpdate))
print('----------------------------Testing Function---------------------------------\n')

# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo_two')
splice.execute('create table foo_two(a int)')
splice.execute('insert into foo_two values(1)')

#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    splice.df('select * from splice.foo_two').show()
    splice.executeUpdate('drop table if exists splice.foo_two')
    try:
        splice.df('select * from splice.foo_two').show()
    except:
        print('table dropped')
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    splice.executeUpdate('drop table splice.foo_two')
    try:
        splice.df('select * from splice.foo_two').show()
    except:
        print('table dropped')
    

count+=1

Function Definition:
    def executeUpdate(self, query_string):
        '''
        execute a dml query:(update,delete,drop,etc)
        :param query_string: (string) SQL Query (eg. SELECT * FROM table1 WHERE column2 > 3)
        :return:
        '''
        return self.context.executeUpdate(query_string)

----------------------------Testing Function---------------------------------

+---+
|  A|
+---+
|  1|
+---+

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reason: userid or password invalid.
table dropped


## Test export

In [14]:
print(inspect.getsource(splice.export))

    def export(self, dataframe, location, compression=False, replicationCount=1, fileEncoding=None,
               fieldSeparator=None,
               quoteCharacter=None):
        '''
        Export a dataFrame in CSV
        :param dataframe:
        :param location: Destination directory
        :param compression: Whether to compress the output or not
        :param replicationCount:  Replication used for HDFS write
        :param fileEncoding: fileEncoding or null, defaults to UTF-8
        :param fieldSeparator: fieldSeparator or null, defaults to ','
        :param quoteCharacter: quoteCharacter or null, defaults to '"'
        :return:
        '''
        return self.context.export(dataframe._jdf, location, compression, replicationCount,
                                   fileEncoding,
                                   fieldSeparator, quoteCharacter)



## Test exportBinary

In [15]:
print('Function Definition:')
print(inspect.getsource(splice.exportBinary))
print('----------------------------Testing Function---------------------------------\n')


Function Definition:
    def exportBinary(self, dataframe, location, compression, e_format):
        '''
        Export a dataFrame in binary format
        :param dataframe:
        :param location: Destination directory
        :param compression: Whether to compress the output or not
        :param e_format: Binary format to be used, currently only 'parquet' is supported
        :return:
        '''
        return self.context.exportBinary(dataframe._jdf, location, compression, e_format)

----------------------------Testing Function---------------------------------



## Test getConnection

In [16]:
print('Function Definition:')
print(inspect.getsource(splice.getConnection))
print('----------------------------Testing Function---------------------------------\n')
#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    x = splice.getConnection()
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    x = splice.getConnection()
print(type(x))
print(x)
count+=1

Function Definition:
    def getConnection(self):
        """
        Return a connection to the database
        """
        return self.context.getConnection()

----------------------------Testing Function---------------------------------

<class 'py4j.java_gateway.JavaObject'>
com.splicemachine.db.impl.jdbc.EmbedConnection40@796609640 (XID = SpliceTransaction[IDLE,null]), (SESSIONID = 1234), (DATABASE = splicedb), (DRDAID = null) 


## Test getSchema

In [17]:
print('Function Definition:')
print(inspect.getsource(splice.getSchema))
print('----------------------------Testing Function---------------------------------\n')


# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo')
splice.execute('create table foo(a int, b varchar(50), c int, primary key(a))')
splice.execute('insert into foo values(1, \'test\', 2)')
splice.execute('insert into foo values(2, \'foo\', 3)')
splice.execute('insert into foo values(3, \'bar\', 12)')
splice.execute('insert into foo values(4, \'name\', 42)')
splice.execute('insert into foo values(5, \'field\', 34)')
splice.execute('insert into foo values(6, \'nan\', 77)')
splice.df('select * from foo').show()


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    x = splice.getSchema('splice.foo')
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    x = splice.getSchema('splice.foo')
print(type(x))
print(x)
count+=1

Function Definition:
    def getSchema(self, schema_table_name):
        """
        Return the schema via JDBC.

        :param schema_table_name: (DF) Table name
        """
        return self.context.getSchema(schema_table_name)

----------------------------Testing Function---------------------------------

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reason: userid or password invalid.
<class 'py4j.java_gateway.JavaObject'>
StructType(StructField(A,IntegerType,true), StructField(B,StringType,true), StructField(C,IntegerType,true))


## Test insert

In [18]:
from pyspark.sql.functions import lit
print('Function Definition:')
print(inspect.getsource(splice.insert))
print(f'----------------------------Testing Function---------------------------------\n')

# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo')
splice.execute('create table foo(a int, b varchar(50), c int, primary key(a))')
splice.execute('insert into foo values(1, \'test\', 2)')
splice.execute('insert into foo values(2, \'foo\', 3)')
splice.execute('insert into foo values(3, \'bar\', 12)')
splice.execute('insert into foo values(4, \'name\', 42)')
splice.execute('insert into foo values(5, \'field\', 34)')
splice.execute('insert into foo values(6, \'nan\', 77)')
splice.df('select * from foo').show()


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    splice.df('select * from splice.foo').show()
    x  = splice.df('select * from foo where a=1')
    x = x.withColumn('A',lit(11))
    splice.insert(x,'splice.foo')
    splice.df('select * from splice.foo').show()
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    x  = splice.df('select * from foo where a=1')
    x = x.withColumn('A',lit(11))
    splice.insert(x,'splice.foo')
    splice.df('select * from splice.foo').show()
    

count+=1

Function Definition:
    def insert(self, dataframe, schema_table_name):
        """
        Insert a dataframe into a table (schema.table).

        :param dataframe: (DF) The dataframe you would like to insert
        :param schema_table_name: (string) The table in which you would like to insert the RDD
        """
        # make sure column names are in the correct case
        dataframe = self.replaceDataframeSchema(dataframe, schema_table_name)
        return self.context.insert(dataframe._jdf, schema_table_name)

----------------------------Testing Function---------------------------------

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo

## Test internalDf: Not sure what's wrong with this one

In [19]:
# print('Function Definition:')
# print(inspect.getsource(splice.internalDf))
# print('----------------------------Testing Function---------------------------------\n')

# # Setup
# splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
# splice.execute('drop table if exists foo')
# splice.execute('create table foo(a int, b varchar(50), c int, primary key(a))')
# splice.execute('insert into foo values(1, \'test\', 2)')
# splice.execute('insert into foo values(2, \'foo\', 3)')
# splice.execute('insert into foo values(3, \'bar\', 12)')
# splice.execute('insert into foo values(4, \'name\', 42)')
# splice.execute('insert into foo values(5, \'field\', 34)')
# splice.execute('insert into foo values(6, \'nan\', 77)')
# splice.df('select * from foo').show()


# #Fails when you don't pass in user/password into pysplicecontext
# if splice: del splice
# try:
#     splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
#     x = splice.internalDf('select * from splice.foo where a=3')
# except Exception as e:
#     auth_functions.append(functions[count])
#     print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
#     splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
#     x = splice.internalDf('select * from splice.foo where a=3')
# print(type(x))
# print(x)
# x.show()
# count+=1

## Test spark_sql_context

In [20]:
import time
print('Function Definition:')
sqlContext = splice.spark_sql_context
print(sqlContext)
print('----------------------------Testing Function---------------------------------\n')


# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo')
splice.execute('create table foo(a int, b varchar(50), c int, primary key(a))')
splice.execute('insert into foo values(1, \'test\', 2)')
splice.execute('insert into foo values(2, \'foo\', 3)')
splice.execute('insert into foo values(3, \'bar\', 12)')
splice.execute('insert into foo values(4, \'name\', 42)')
splice.execute('insert into foo values(5, \'field\', 34)')
splice.execute('insert into foo values(6, \'nan\', 77)')
splice.df('select * from foo').show()



if splice: del splice
if sqlContext: del sqlContext
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    sqlContext = splice.spark_sql_context
    x = splice.df('select * from splice.foo')
    x.registerTempTable('test')
    print('should match above')
    try:
        sqlContext.sql('select * from test').show()
    except:
        print('takes a failure and exception and rerun of cell to get table registered')
        
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    sqlContext = splice.spark_sql_context
    x = splice.df('select * from splice.foo')
    x.registerTempTable('test')
    print('should match above')
    sqlContext.sql('select * from test').show()

count+=1

Function Definition:
----------------------------Testing Function---------------------------------

+---+-----+---+
|  A|    B|  C|
+---+-----+---+
|  1| test|  2|
|  2|  foo|  3|
|  3|  bar| 12|
|  4| name| 42|
|  5|field| 34|
|  6|  nan| 77|
+---+-----+---+

should match above
takes a failure and exception and rerun of cell to get table registered


## Test tableExists


In [21]:
## print('Function Definition:')
print(inspect.getsource(splice.tableExists))
print('----------------------------Table testing on---------------------------------\n')

# Setup
splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo_two')
splice.execute('create table foo_two(a int)')
splice.execute('insert into foo_two values(1)')
splice.df('select * from foo_two').show()

if splice: del splice
try:
    print('----------------------------Testing tableExists with table that exists---------------------------------\n')
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    print(splice.tableExists('splice.foo_two'))
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    a = splice.tableExists('splice.foo_two')
    try:
        assert(a)
    except:
        print(f'Should have been True, got {a}')
    print(f'Table exists: {a}')
    

    

# On table that does not exist
splice.dropTable('splice.foo_two')
if splice: del splice
try:
    print('----------------------------Testing tableExists with table that does NOT exists---------------------------------\n')
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    print(splice.tableExists('splice.foo_two'))
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    a = splice.tableExists('splice.foo_two')
    try:
        assert(not a)
    except:
        print(f'Should have been False got {a}')
    print(f'Table exists: {a}')
    
count+=1

    def tableExists(self, schema_table_name):
        """
        Check whether or not a table exists

        :param schema_table_name: (string) Table Name
        """
        return self.context.tableExists(schema_table_name)

----------------------------Table testing on---------------------------------

+---+
|  A|
+---+
|  1|
+---+

----------------------------Testing tableExists with table that exists---------------------------------

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reason: userid or password invalid.
Table exists: True
----------------------------Testing tableExists with table that does NOT exists---------------------------------

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reason: userid or password invalid.
Table exists: False


## Test truncateTable

In [22]:
print('Function Definition:')
print(inspect.getsource(splice.truncateTable))
print('----------------------------Testing Function---------------------------------\n')

splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo_two')
splice.execute('create table foo_two(a int)')
splice.execute('insert into foo_two values(1)')
splice.execute('insert into foo_two values(4)')
splice.execute('insert into foo_two values(555)')
splice.execute('insert into foo_two values(33)')
splice.execute('insert into foo_two values(2)')
splice.df('select * from foo_two').show()


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    splice.truncateTable('splice.foo_two')
    splice.df('select * from foo_two').show()
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    splice.truncateTable('splice.foo_two')
    splice.df('select * from foo_two').show()

    
count+=1

Function Definition:
    def truncateTable(self, schema_table_name):
        """
        truncate a table
        :param schema_table_name: the full table name in the format "schema.table_name" which will be truncated
        :return:
        """
        return self.context.truncateTable(schema_table_name)

----------------------------Testing Function---------------------------------

+---+
|  A|
+---+
|  1|
|  4|
|555|
| 33|
|  2|
+---+

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection authentication failure occurred.  Reason: userid or password invalid.
+---+
|  A|
+---+
+---+



## Test update: 
### Note: table must have primary key to call a update
#### Also Note: You cannot update the primary key (as expected)


In [23]:
print('Function Definition:')
print(inspect.getsource(splice.update))
print('----------------------------Testing Function---------------------------------\n')

splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo_two')
splice.execute('create table foo_two(a int, b int, primary key(a))')
splice.execute('insert into foo_two values(1, 2)')
splice.execute('insert into foo_two values(3, 4)')
splice.execute('insert into foo_two values(5, 6)')
splice.df('select * from foo_two').show()


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    x = splice.df('select * from splice.foo_two')
    print('\nWhat the table should look like:')
    x = x.withColumn('B', x['B']*2)
    x.show()
    splice.update(x, 'splice.foo_two')
    print('The table below should match the table above:')
    splice.df('select * from foo_two').show()
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    x = splice.df('select * from splice.foo_two')
    print('\nWhat the table should look like:')
    x = x.withColumn('B', x['B']*2)
    x.show()
    splice.update(x, 'splice.foo_two')
    print('The table below should match the table above:')
    splice.df('select * from foo_two').show()
    
count+=1

Function Definition:
    def update(self, dataframe, schema_table_name):
        """
        Update data from a dataframe for a specified schema_table_name (schema.table).
        The keys are required for the update and any other columns provided will be updated
        in the rows.

        :param dataframe: (DF) The dataframe you would like to update
        :param schema_table_name: (string) Splice Machine Table
        """
        # make sure column names are in the correct case
        dataframe = self.replaceDataframeSchema(dataframe, schema_table_name)
        return self.context.update(dataframe._jdf, schema_table_name)

----------------------------Testing Function---------------------------------

+---+---+
|  A|  B|
+---+---+
|  1|  2|
|  3|  4|
|  5|  6|
+---+---+


What the table should look like:
+---+---+
|  A|  B|
+---+---+
|  1|  4|
|  3|  8|
|  5| 12|
+---+---+

NOTE: Failed without user/password in JDBC URL:
 java.sql.SQLNonTransientConnectionException: Connection au

## Test upsert

In [24]:
print('Function Definition:')
print(inspect.getsource(splice.upsert))
print('----------------------------Testing Function---------------------------------\n')

splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
splice.execute('drop table if exists foo_two')
splice.execute('create table foo_two(a int, b int, primary key(a))')
splice.execute('insert into foo_two values(1, 2)')
splice.execute('insert into foo_two values(3, 4)')
splice.execute('insert into foo_two values(5, 6)')
splice.df('select * from foo_two').show()


#Fails when you don't pass in user/password into pysplicecontext
if splice: del splice
try:
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb')
    x = splice.df('select * from splice.foo_two')
    print('\nThe table should have the following rows appended:')
    x = x.withColumn('A', x['A']*2)
    x.show()
    splice.upsert(x, 'splice.foo_two')
    print('The table below should have the rows from above appended:')
    splice.df('select * from foo_two').show()
except Exception as e:
#     auth_functions.append(functions[count])
    print('NOTE: Failed without user/password in JDBC URL:\n', e.java_exception)
    splice = PySpliceContext(spark, f'jdbc:splice://{jdbc_host}:1527/splicedb;user=splice;password=kolj$6drornUrk')
    x = splice.df('select * from splice.foo_two')
    print('\nThe table should have the following rows appended:')
    x = x.withColumn('A', x['A']*2)
    x.show()
    splice.upsert(x, 'splice.foo_two')
    print('The table below should have the rows from above appended:')
    splice.df('select * from foo_two').show()
    
count+=1

Function Definition:
    def upsert(self, dataframe, schema_table_name):
        """
        Upsert the data from a dataframe into a table (schema.table).

        :param dataframe: (DF) The dataframe you would like to upsert
        :param schema_table_name: (string) The table in which you would like to upsert the RDD
        """
        # make sure column names are in the correct case
        dataframe = self.replaceDataframeSchema(dataframe, schema_table_name)
        return self.context.upsert(dataframe._jdf, schema_table_name)

----------------------------Testing Function---------------------------------

+---+---+
|  A|  B|
+---+---+
|  1|  2|
|  3|  4|
|  5|  6|
+---+---+


The table should have the following rows appended:
+---+---+
|  A|  B|
+---+---+
|  2|  2|
|  6|  4|
| 10|  6|
+---+---+

The table below should have the rows from above appended:
+---+---+
|  A|  B|
+---+---+
|  1|  2|
|  2|  2|
|  3|  4|
|  5|  6|
|  6|  4|
| 10|  6|
+---+---+

